In [1]:
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing
import warnings
warnings.filterwarnings("ignore")
import random
from datetime import datetime
import time
from glob import glob
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from PIL import Image
import torch
import numpy as np
import pandas as pd


#### Configurations

In [2]:
class CFG:
    seed = 42
    img_height = 640
    img_width = 640
    train_batch_size = 4
    val_batch_size = 4
    verbose = True
    verbose_step = 1
    lr = 0.0001
    n_epochs  = 1
    folds = 5
    random_state = 42
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True)
seed_everything(CFG.seed)

In [3]:
df = pd.read_csv('data/Train.csv')
unique_classes = df['class'].unique()
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
print(class_mapping)


{'Trophozoite': 0, 'NEG': 1, 'WBC': 2}


In [4]:
# Set the data directory
ROOT_DIR = Path(os.getcwd())
DATA_DIR = ROOT_DIR / 'data'
IMGS_DIR = DATA_DIR / 'images'

# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

In [5]:
unique_classes = train['class'].unique()
full_label_dict = {cls: idx for idx, cls in enumerate(unique_classes)}
full_label_dict

{'Trophozoite': 0, 'NEG': 1, 'WBC': 2}

In [6]:
unique_classes = train[train['class']!='NEG']['class'].unique()
label_dict = {cls: idx for idx, cls in enumerate(unique_classes)}
label_dict

{'Trophozoite': 0, 'WBC': 1}

#### Cross-validation Setup

In [7]:
# create data for different folds

# Step 1: Group by Image_ID and aggregate class labels into lists
train['new_class'] = train['class'].map(full_label_dict)
grouped = train.groupby('Image_ID')['new_class'].apply(list).reset_index()

grouped['Trophozoite'] = -1
grouped['WBC'] = -1
grouped['NEG'] = -1

# input 1 if the label is in that image else 0
all_labels_list = (list(grouped['new_class'].values))
for train_index, label_List in enumerate(all_labels_list):
    unique_labels = list(set(label_List))
    for label_index in range(len(unique_labels)):
        label = int(unique_labels[label_index])
        if label == 0:
            grouped.loc[train_index, 'Trophozoite'] = 1

        elif label == 1:
            grouped.loc[train_index, 'WBC'] = 1

        elif label == 2:
            grouped.loc[train_index, 'NEG'] = 1

        
X = grouped[['Image_ID']]
grouped['fold'] = -1
mskf = MultilabelStratifiedKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.random_state)
for i_fold, (train_index, test_index) in enumerate(mskf.split(X, grouped[["Trophozoite", "WBC", "NEG"]])):
    grouped.loc[test_index, "fold"] = i_fold 


# create image_path for grouped_data
grouped['image_path'] = [Path(str(IMGS_DIR) + '/' + x) for x in grouped.Image_ID]

# drop duplicates rows for test
test = test.drop_duplicates(subset=['Image_ID'], ignore_index=True)
test['image_path'] = [Path(str(IMGS_DIR) + '/' + x) for x in test.Image_ID]      
        

#### BOX Processing Function


In [8]:
# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):

    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"


    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    if np.isnan(class_id):
        with open(label_file, 'a') as f:
            f.write(f"")
    else:
        with open(label_file, 'a') as f:
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    # convert the dataframe to a dictionary
    dataframe = dataframe.to_dict('records')
    for i in tqdm(range(len(dataframe))):
        save_yolo_annotation(dataframe[i])



    # with multiprocessing.Pool(1) as pool:
    #     list(tqdm(pool.imap(save_yolo_annotation, dataframe.head().to_dict('records')), total=len(dataframe.head())))

#### Cross-Validation Implementation and Bounding Box Preprocessing

In [9]:
# Add an image_path column
train['image_path'] = [Path(str(IMGS_DIR) + '/' + x) for x in train.Image_ID]

# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map(label_dict)

In [10]:
# for fold in range(CFG.folds):
for fold in range(0, 1):
    # images
    TRAIN_IMAGES_DIR = ROOT_DIR /'train' / 'images' / f'fold_{fold + 1}'
    VAL_IMAGES_DIR = ROOT_DIR / 'val' / 'images' / f'fold_{fold + 1}'
    TEST_IMAGES_DIR = ROOT_DIR / 'test' / 'images'

    # labels
    TRAIN_LABELS_DIR = ROOT_DIR / 'train' / 'labels' / f'fold_{fold + 1}'
    VAL_LABELS_DIR = ROOT_DIR / 'val' / 'labels' / f'fold_{fold + 1}'
    TEST_LABELS_DIR = ROOT_DIR / 'test' / 'labels'


    DIRS = [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR]


    # get the train and val for that fold
    train_fold = grouped[grouped['fold'] != fold ].reset_index(drop=True)
    val_fold = grouped[grouped['fold'] == fold].reset_index(drop=True)

    # train_fold_idxs = train_fold['Image_ID'].tolist()
    # val_fold_idxs = val_fold['Image_ID'].tolist()

    # if code is running for the first time, then create a folder for test 
    if fold == 0:
        DIRS = [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_IMAGES_DIR, TEST_LABELS_DIR]
        


    # Create necessary directories
    for DIR in DIRS:
        if DIR.exists():
            shutil.rmtree(DIR)
        DIR.mkdir(parents=True, exist_ok=True)
       

    # Copy train, val, and test images to their respective dirs
    for img in tqdm(train_fold.image_path.unique()):
        shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])
    print(f'Copied train file for fold{fold+1} to folder')

    for img in tqdm(val_fold.image_path.unique()):
        shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])
    print(f'Copied val file for fold{fold+1} to folder')

    # if code is running for the first time, then move the images to the folder
    # if fold == 0:
    #     for img in tqdm(test.image_path.unique()):
    #         shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])
    #     print(f'Copied test file for first fold to folder')


    X_train = train[train.Image_ID.isin(train_fold.Image_ID)].reset_index(drop=True)
    X_val = train[train.Image_ID.isin(val_fold.Image_ID)].reset_index(drop=True)


    # Save train and validation labels to their respective dirs
    process_dataset(X_train, TRAIN_LABELS_DIR)
    process_dataset(X_val, VAL_LABELS_DIR)

#------------------------------- Took about 55 minutes in running this code

  0%|          | 0/2198 [00:00<?, ?it/s]

Copied train file for fold1 to folder


  0%|          | 0/549 [00:00<?, ?it/s]

Copied val file for fold1 to folder


  0%|          | 0/18949 [00:00<?, ?it/s]

  0%|          | 0/4581 [00:00<?, ?it/s]

In [ ]:
# Create a data.yaml file required by YOLO
class_names = train['class'].unique().tolist()
class_names = [label for label in class_names if label != 'NEG']
num_classes = len(class_names)


# for fold in range(CFG.folds):
for fold in range(0, 1):
    # images
    TRAIN_IMAGES_DIR = ROOT_DIR / 'train' / 'images' / f'fold_{fold + 1}'
    VAL_IMAGES_DIR = ROOT_DIR / 'val' / 'images' / f'fold_{fold + 1}'

    data_yaml = {
        'train': str(TRAIN_IMAGES_DIR),
        'val': str(VAL_IMAGES_DIR),
        'nc': num_classes,
        'names': class_names
    }

    os.makedirs(ROOT_DIR / 'yaml', exist_ok=True)

    # Save the data.yaml file
    yaml_path = ROOT_DIR / 'yaml' / f'fold_{fold+1}.yaml'
    with open(yaml_path, 'w') as file:
        yaml.dump(data_yaml, file, default_flow_style=False)

In [13]:
seed_everything(CFG.seed)

# Load a YOLO pretrained model
model = YOLO('yolov8s-p6.yaml').load('yolov8s.pt')

# using https://github.com/DmitriyKras/YOLOv8-for-small-objects?tab=readme-ov-file
# model = YOLO(model=r'C:\WORKS\DSCompetitions\Lacuna_Malaria\yolov8-p2_woP5_sandwich.yaml')
# yolov8s-p2



model.train(
    data=yaml_path,          # Path to the dataset configuration
    time=1,                 # Number of epochs - 2epochs
    imgsz=1100,               # Image size - 604image size
    batch=16,                  # Batch size - order of 32, 32, 16 
    device=CFG.device,               # Use the first GPU (0 for the first GPU)
    patience=20,  # Number of epochs with no improvement after which training will stop
    seed = CFG.seed
)


Transferred 125/467 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.28  Python-3.10.15 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolov8s-p6.yaml, data=c:\WORKS\DSCompetitions\Lacuna_Malaria\yaml\fold_1.yaml, epochs=100, time=1, patience=20, batch=16, imgsz=1100, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train6, exist_ok=False, pretrained=yolov8s.pt, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment

train: Scanning C:\WORKS\DSCompetitions\Lacuna_Malaria\train\labels\fold_1.cache... 2198 images, 549 backgrounds, 0 corrupt: 100%|██████████| 2198/2198 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\WORKS\DSCompetitions\Lacuna_Malaria\val\labels\fold_1.cache... 549 images, 139 backgrounds, 0 corrupt: 100%|██████████| 549/549 [00:00<?, ?it/s]


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 75 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 1152 train, 1152 val
Using 8 dataloader workers
Logging results to runs\detect\train6
Starting training for 1 hours...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        549       4442       0.79      0.835      0.852      0.434


EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 64, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

84 epochs completed in 0.709 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 36.1MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 36.1MB

Validating runs\detect\train6\weights\best.pt...
WARNING  validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.28  Python-3.10.15 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8s-p6 summary (fused): 220 layers, 17,857,436 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.12it/s]


                   all        549       4442      0.798      0.835      0.856      0.444
           Trophozoite        403       3134      0.686      0.715      0.738      0.303
                   NEG        315       1308      0.911      0.955      0.974      0.586
Speed: 0.3ms preprocess, 2.0ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001FEA525AFE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

#### Standard Inference

In [ ]:
# Make predictions on test images and save them to a CSV file
VAL_IMAGES_DIR = ROOT_DIR / 'val' / 'images' / 'fold_1'
seed_everything(CFG.seed)
image_files = os.listdir(VAL_IMAGES_DIR)
all_data = []
neg_prediction = 0 

# create submission (subs) directory
os.makedirs(ROOT_DIR / 'subs', exist_ok=True)

MODEL_PATH = ROOT_DIR / 'runs\detect' # path to where all trained logs (images, weights) are saved

best_trained_models = []
for dirpath, dirnames, filenames in (os.walk(MODEL_PATH)):
    trained_models = [os.path.join(dirpath, file) for file in filenames if 'best.pt' in file]
    if trained_models:
        best_trained_models.extend(trained_models) # path to best_trained_models

model = YOLO(best_trained_models[0])


for image_file in tqdm(image_files):
    img_path = os.path.join(VAL_IMAGES_DIR, image_file)
    results = model(img_path, verbose=False)
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    confidences = results[0].boxes.conf.tolist()
    names = results[0].names
    
    if not boxes:
        neg_prediction = neg_prediction +   1 
        all_data.append({
            'Image_ID': image_file,
            'class': 'NEG',
            'confidence': 1.0,
            'ymin': 0,
            'xmin': 0,
            'ymax': 0,
            'xmax': 0
        })
    else:
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[int(cls)]

            all_data.append({
                'Image_ID': image_file,
                'class': detected_class,
                'confidence': conf,
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })
print(f'========== Total Negative Images {neg_prediction}================')
# Convert the results to a DataFrame and save it
sub = pd.DataFrame(all_data)
sub.to_csv(ROOT_DIR / 'subs/yolov8-p6.csv')